<a href="https://colab.research.google.com/github/jdz014/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/LS_DS12_214_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [X] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [X] Begin with baselines for classification.
- [X] Use scikit-learn for logistic regression.
- [X] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [X] Get your model's test accuracy. (One time, at the end.)
- [X] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [X] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.

In [140]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [141]:
df.dtypes

Burrito            object
Date               object
Yelp              float64
Google            float64
Chips              object
Cost              float64
Hunger            float64
Mass (g)          float64
Density (g/mL)    float64
Length            float64
Circum            float64
Volume            float64
Tortilla          float64
Temp              float64
Meat              float64
Fillings          float64
Meat:filling      float64
Uniformity        float64
Salsa             float64
Synergy           float64
Wrap              float64
Unreliable         object
NonSD              object
Beef               object
Pico               object
Guac               object
Cheese             object
Fries              object
Sour cream         object
Pork               object
Chicken            object
Shrimp             object
Fish               object
Rice               object
Beans              object
Lettuce            object
Tomato             object
Bell peper         object
Carrots     

In [0]:
# Changing 'Date' from object to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [0]:
# Checking if change worked
df.dtypes

In [0]:
# Create train data for 2016 & earlier
train = df[df['Date'].dt.year <= 2016]

# Create validation data for 2017
val = df[df['Date'].dt.year == 2017]

# Create test data for 2018 & later
test = df[df['Date'].dt.year >= 2018]

Begin with baselines for classification.

In [145]:
# Determine majority class
target = 'Great'
y_train = train[target]

y_train.value_counts(normalize=True)

False    0.590604
True     0.409396
Name: Great, dtype: float64

In [0]:
majority_class = y_train.mode()[0]

In [147]:
from sklearn.metrics import accuracy_score

# Training accuracy of majority class baseline(base rate)
y_pred = [majority_class] * len(y_train)

accuracy_score(y_train, y_pred)

0.5906040268456376

In [148]:
# Validation accuracy of majority class baseline 
y_val = val[target]
y_pred = [majority_class] * len(y_val)

accuracy_score(y_val, y_pred)

0.5529411764705883

Use scikit-learn for logistic regression.

In [149]:
# Sorting non-numeric columns by unique
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
Chile relleno,4,1,x,4,NaT,NaT
Ham,1,1,x,1,NaT,NaT
Zucchini,1,1,x,1,NaT,NaT
Nopales,4,1,x,4,NaT,NaT
Lobster,1,1,x,1,NaT,NaT
Egg,4,1,x,4,NaT,NaT
Carrots,1,1,x,1,NaT,NaT
Bacon,3,1,x,3,NaT,NaT
Sushi,2,1,x,2,NaT,NaT
Unreliable,27,1,x,27,NaT,NaT


In [150]:
# Checking contents of column
train['Burrito'].value_counts()

California     118
Other          110
Asada           35
Surf & Turf     21
Carnitas        14
Name: Burrito, dtype: int64

In [0]:
# Dropping categorical columns with high cardinality
target = 'Great'
high_cardinality = ['Date']
features = train.columns.drop([target] + high_cardinality)

In [0]:
# Creating X_train/val/test and y_train/val/test with the useful features
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

X_val = val[features]
y_val = val[target]

In [0]:
# One-hot encoding
import category_encoders as ce

# Using fit_transform on X_train
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)

# Using encoder.transform on X_val (no .fit)
X_val_encoded = encoder.transform(X_val)

# Using encoder.transform on X_test (no .fit)
X_test_encoded = encoder.transform(X_test)

In [0]:
# Impute missing values
from sklearn.impute import SimpleImputer

# Using fit_transform on X_train_encoded
imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train_encoded)

# Using imputer.transform on X_val_encoded (no .fit)
X_val_imputed = imputer.transform(X_val_encoded)

# Using imputer.transform on X_val_encoded (no .fit)
X_test_imputed = imputer.transform(X_test_encoded)

In [0]:
# Scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Using fit.transform on X_val_imputed
X_train_scaled = scaler.fit_transform(X_train_imputed)

# Using scaler.transform on X_val_imputed (no .fit)
X_val_scaled = scaler.transform(X_val_imputed)

# Using scaler.transform on X_val_imputed (no .fit)
X_test_scaled = scaler.transform(X_test_imputed)

In [179]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_scaled, y_train)
model.fit(X_val_scaled, y_val)
model.fit(X_test_scaled, y_test)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Get your model's validation accuracy. (Multiple times if you try multiple iterations.)

In [180]:
print('Validation Accuracy', model.score(X_train_scaled, y_train))
print('Validation Accuracy', model.score(X_val_scaled, y_val))

Validation Accuracy 0.7785234899328859
Validation Accuracy 0.6941176470588235


Get your model's test accuracy. (One time, at the end.)

In [176]:
print('Test Accuracy', model.score(X_test_scaled, y_test))

Test Accuracy 1.0
